In [1]:
import pandas as pd

In [2]:
excelfile = "excel location"

In [3]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

In [10]:
df = pd.read_csv(excelfile, sep="|", error_bad_lines=False)

In [11]:
df.drop(df[(df.HYBRID_VEHICLE_FLAG == 'N') &
            (df.ELECTRIC_VEHICLE_FLAG == 'N')].index, inplace=True)
df['MODEL_YEAR'].fillna(0, inplace=True)
df['VIN'].fillna(0, inplace=True)
# df.drop(df[(df.MODEL_YEAR < 2019)].index, inplace=True)

df.drop(df[(df.VIN == 0)].index, inplace=True)

df.drop(df.columns.difference([
  'VIN',
  'MODEL',
  'MODEL_YEAR',
  'MAKE',
  'ELECTRIC_VEHICLE_FLAG',
  'HYBRID_VEHICLE_FLAG',
  'FUEL_TYPE',
  'VEHICLE_TYPE',
  'NET_WEIGHT']), 1, inplace=True)
df = trim_all_columns(df)
df["MODEL_YEAR"] = df["MODEL_YEAR"].astype(int)
df["MODEL"] = df["MODEL"].str.upper()
df["MAKE"]= df["MAKE"].str.upper()

In [ ]:
df.to_excel("output.xlsx")

In [269]:
zevadf = df.drop(df.columns.difference([
  'MODEL',
  'MAKE',
'MODEL']), 1)
zevadf['MakeModel'] = zevadf['MAKE'].str.cat(zevadf['MODEL'],sep=" ")
zevadf = zevadf.drop(zevadf.columns.difference([
  'MakeModel']), 1)
zevadf.drop_duplicates(inplace=True)
zevadf = zevadf.sort_values('MakeModel')

In [76]:
zevadf.to_excel("no_years_dropped.xlsx")

In [216]:
j_list_df=pd.read_excel("jmasterlist.xlsx", header=1)

In [270]:
combined_new_vehicles = pd.merge(left=zevadf, right=j_list_df, left_on="MakeModel", right_on="COMBINED", how='outer')
combined_new_vehicles.columns=['ZEVA_parser', 'J_list']
combined_new_vehicles.columns=['ZEVA_parser', 'J_list']
combined_new_vehicles['all'] = combined_new_vehicles['ZEVA_parser']
combined_new_vehicles.loc[combined_new_vehicles['all'].isnull(), 'all'] = combined_new_vehicles['J_list']


In [272]:
combined_new_vehicles.to_excel("differences_all_vehicles.xlsx")